In [4]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [5]:
# Hyper Parameters
EPOCH = 1
BATCH_SIZE = 64
TIME_STEP = 28
INPUT_SIZE = 28
LR = 0.01
DOWNLOAD_MNIST = False

In [6]:
train_data = dsets.MNIST(root='./mnist', train=True, transform=transforms.ToTensor(), download=DOWNLOAD_MNIST)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = dsets.MNIST(root='./mnist/', train=False, transform=transforms.ToTensor())
test_x = Variable(test_data.test_data, volatile=True).type(torch.FloatTensor)[:2000] / 255
test_y = test_data.test_labels.numpy().squeeze()[:2000]

D:\python\python 3.5\lib\site-packages\ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


In [7]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size=64,
            num_layers=1,
            batch_first=True,  # (batch,timepstep,input)
        )

        self.out = nn.Linear(64, 10)

    def forward(self, x):
        r_out, (h_n, h_c) = self.rnn(x, None)  # h_n,h_c 一个主线输出，一个控制输出
        out = self.out(r_out[:, -1, :])  # (batch,timestep,input)
        return out


rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)

In [8]:
optimzier = torch.optim.Adam(rnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [10]:
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        b_x = Variable(x.view(-1, 28, 28))  # x.view = reshape,(batch,timestep,input)
        b_y = Variable(y)
        output = rnn(b_x)
        loss = loss_func(output, b_y)
        optimzier.zero_grad()
        loss.backward()
        optimzier.step()

        # 每过50步就用测试集测试一下，没问题，因为测试集没有经过反向传播，仅测试
        if step % 50 == 0:
            test_output = rnn(test_x)
            # pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
            # accuracy = sum(pred_y == test_y) / test_y.size()
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y).astype(int).sum()) / float(test_y.size)
            print('Epoch:', epoch, '| train loss:%.4f' % loss.data[0], '| test accuracy:%.2f' % accuracy)

    # print 10 predictions from test data
    test_output = rnn(test_x[:10].view(-1, 28, 28))
    pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
    print(pred_y, 'predictin number')
    print(test_y[:10], 'real number')

D:\python\python 3.5\lib\site-packages\ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch:

0

| train loss:2.2854

| test accuracy:0.20

...

| test accuracy:0.95

[7 2 1 0 4 1 4 9 6 9]

predictin number

[7 2 1 0 4 1 4 9 5 9]

real number